# 1일차 Review : 같은 문제 다시 풀어보기

    학습은 10번으로!

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()

In [3]:
x = iris.data
y = iris.target

In [4]:
x.shape, y.shape

((150, 4), (150,))

In [5]:
#####################
# 라이브러리 불러오기
#####################
import tensorflow as tf
from tensorflow import keras

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical       #  y에 대한 one-hot encoding

In [6]:
#######################
# 어떤 전처리 작업 필요
# Hint : y.shape
#######################
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=2022,test_size=0.2)

In [7]:
y_len = len(set(y_train))

In [8]:
set(y_train)

{0, 1, 2}

In [9]:
y_train = to_categorical(y_train, y_len)
y_test = to_categorical(y_test,y_len)

In [10]:
x_train.shape, y_train.shape

((120, 4), (120, 3))

In [11]:
########
# 모델링
########
keras.backend.clear_session()

il = keras.layers.Input(shape = (4,))
ol = keras.layers.Dense(3, activation = 'softmax')(il)

model = keras.models.Model(il, ol)

model.compile(loss= 'categorical_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 3)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [12]:
###########
# 모델 학습
###########

model.fit(x_train, y_train, epochs=10, verbose = 1, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 130ms/step - loss: 2.1498 - accuracy: 0.0938 - val_loss: 2.5104 - val_accuracy: 0.1250
Epoch 2/10
3/3 [==============================] - 0s 24ms/step - loss: 2.1319 - accuracy: 0.0833 - val_loss: 2.4779 - val_accuracy: 0.0833
Epoch 3/10
3/3 [==============================] - 0s 21ms/step - loss: 2.1144 - accuracy: 0.0312 - val_loss: 2.4468 - val_accuracy: 0.0833
Epoch 4/10
3/3 [==============================] - 0s 21ms/step - loss: 2.0949 - accuracy: 0.0312 - val_loss: 2.4182 - val_accuracy: 0.0833
Epoch 5/10
3/3 [==============================] - 0s 22ms/step - loss: 2.0775 - accuracy: 0.0312 - val_loss: 2.3906 - val_accuracy: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 0s 24ms/step - loss: 2.0606 - accuracy: 0.0312 - val_loss: 2.3643 - val_accuracy: 0.0000e+00
Epoch 7/10
3/3 [==============================] - 0s 20ms/step - loss: 2.0451 - accuracy: 0.0312 - val_loss: 2.3383 - val_accuracy: 0.0000e+00
Epoch 8/10
3/3

In [46]:
###########
# 모델 예측
###########
y_pred = model.predict(x_test)

y_pred[:5]


array([[0.00232528, 0.2756977 , 0.72197706],
       [0.00122932, 0.30973873, 0.68903196],
       [0.90051717, 0.08735523, 0.01212763],
       [0.0059493 , 0.40180933, 0.59224135],
       [0.94532466, 0.04870639, 0.00596897]], dtype=float32)

In [47]:
y_pred[:5].argmax(axis=1).reshape(-1,1)

array([[2],
       [2],
       [0],
       [2],
       [0]], dtype=int64)

In [48]:
y_test[:5]

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

# 히든 레이어 추가하여 모델링! (Functional API)

    히든 레이어 2개를 추가하여 다시 모델링!
    히든 레이어 노드 수 : 32
    히든 레이어 activation = 'relu'

In [55]:
########
# 모델링
########
keras.backend.clear_session()

il = keras.layers.Input(shape = (4,))
hl = keras.layers.Dense(32,activation = 'relu')(il)
ol = keras.layers.Dense(3, activation = 'softmax')(hl)

model = keras.models.Model(il, ol)

model.compile(loss= 'categorical_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 259
Trainable params: 259
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

In [57]:
es = EarlyStopping(monitor='val_loss',         # 무엇을 관찰할래?
                  min_delta = 0,               # 0이면 최소한 나빠지지만 않으면 괜찮아
                  patience = 3,                # monitor가 개선되지 않는 거를 몇번 참을래
                  verbose = 1,                 # Early Stopping 된 것을 보여줌     
                  restore_best_weights=True)   # 중요! 최적의 학습 가중치를 모델에 전달

In [58]:
###########
# 모델 학습
###########


model.fit(x_train, y_train, epochs=500, verbose = 1, validation_split=0.2, callbacks=[es])

Epoch 1/500
3/3 [==============================] - 1s 98ms/step - loss: 2.9991 - accuracy: 0.3229 - val_loss: 2.3925 - val_accuracy: 0.4167
Epoch 2/500
3/3 [==============================] - 0s 20ms/step - loss: 2.7535 - accuracy: 0.3229 - val_loss: 2.1876 - val_accuracy: 0.4167
Epoch 3/500
3/3 [==============================] - 0s 19ms/step - loss: 2.5056 - accuracy: 0.3229 - val_loss: 1.9943 - val_accuracy: 0.4167
Epoch 4/500
3/3 [==============================] - 0s 19ms/step - loss: 2.2811 - accuracy: 0.3229 - val_loss: 1.8131 - val_accuracy: 0.4167
Epoch 5/500
3/3 [==============================] - 0s 22ms/step - loss: 2.0715 - accuracy: 0.3229 - val_loss: 1.6465 - val_accuracy: 0.4167
Epoch 6/500
3/3 [==============================] - 0s 23ms/step - loss: 1.8773 - accuracy: 0.3229 - val_loss: 1.4995 - val_accuracy: 0.4167
Epoch 7/500
3/3 [==============================] - 0s 19ms/step - loss: 1.6987 - accuracy: 0.3229 - val_loss: 1.3761 - val_accuracy: 0.4167
Epoch 8/500
3/3 [===

3/3 [==============================] - 0s 25ms/step - loss: 0.8145 - accuracy: 0.5208 - val_loss: 0.7727 - val_accuracy: 0.6667
Epoch 60/500
3/3 [==============================] - 0s 19ms/step - loss: 0.8101 - accuracy: 0.5208 - val_loss: 0.7659 - val_accuracy: 0.6667
Epoch 61/500
3/3 [==============================] - 0s 18ms/step - loss: 0.8043 - accuracy: 0.5208 - val_loss: 0.7612 - val_accuracy: 0.6667
Epoch 62/500
3/3 [==============================] - 0s 19ms/step - loss: 0.7994 - accuracy: 0.5208 - val_loss: 0.7557 - val_accuracy: 0.6667
Epoch 63/500
3/3 [==============================] - 0s 19ms/step - loss: 0.7943 - accuracy: 0.5312 - val_loss: 0.7515 - val_accuracy: 0.6667
Epoch 64/500
3/3 [==============================] - 0s 19ms/step - loss: 0.7896 - accuracy: 0.5729 - val_loss: 0.7467 - val_accuracy: 0.6667
Epoch 65/500
3/3 [==============================] - 0s 18ms/step - loss: 0.7850 - accuracy: 0.5729 - val_loss: 0.7430 - val_accuracy: 0.6250
Epoch 66/500
3/3 [========

3/3 [==============================] - 0s 17ms/step - loss: 0.5956 - accuracy: 0.7604 - val_loss: 0.5548 - val_accuracy: 0.7500
Epoch 118/500
3/3 [==============================] - 0s 20ms/step - loss: 0.5928 - accuracy: 0.7604 - val_loss: 0.5526 - val_accuracy: 0.7500
Epoch 119/500
3/3 [==============================] - 0s 18ms/step - loss: 0.5902 - accuracy: 0.7604 - val_loss: 0.5496 - val_accuracy: 0.7500
Epoch 120/500
3/3 [==============================] - 0s 20ms/step - loss: 0.5884 - accuracy: 0.7708 - val_loss: 0.5487 - val_accuracy: 0.7500
Epoch 121/500
3/3 [==============================] - 0s 24ms/step - loss: 0.5850 - accuracy: 0.7604 - val_loss: 0.5450 - val_accuracy: 0.7500
Epoch 122/500
3/3 [==============================] - 0s 20ms/step - loss: 0.5823 - accuracy: 0.7604 - val_loss: 0.5423 - val_accuracy: 0.7917
Epoch 123/500
3/3 [==============================] - 0s 21ms/step - loss: 0.5806 - accuracy: 0.7708 - val_loss: 0.5382 - val_accuracy: 0.8333
Epoch 124/500
3/3 [=

Epoch 175/500
3/3 [==============================] - 0s 23ms/step - loss: 0.4712 - accuracy: 0.9583 - val_loss: 0.4382 - val_accuracy: 0.8750
Epoch 176/500
3/3 [==============================] - 0s 19ms/step - loss: 0.4691 - accuracy: 0.9583 - val_loss: 0.4362 - val_accuracy: 0.8750
Epoch 177/500
3/3 [==============================] - 0s 19ms/step - loss: 0.4677 - accuracy: 0.9583 - val_loss: 0.4353 - val_accuracy: 0.8750
Epoch 178/500
3/3 [==============================] - 0s 21ms/step - loss: 0.4661 - accuracy: 0.9583 - val_loss: 0.4328 - val_accuracy: 0.8750
Epoch 179/500
3/3 [==============================] - 0s 21ms/step - loss: 0.4644 - accuracy: 0.9583 - val_loss: 0.4311 - val_accuracy: 0.8750
Epoch 180/500
3/3 [==============================] - 0s 19ms/step - loss: 0.4630 - accuracy: 0.9583 - val_loss: 0.4313 - val_accuracy: 0.8750
Epoch 181/500
3/3 [==============================] - 0s 19ms/step - loss: 0.4607 - accuracy: 0.9583 - val_loss: 0.4304 - val_accuracy: 0.8750
Epoch 

Epoch 233/500
3/3 [==============================] - 0s 30ms/step - loss: 0.3811 - accuracy: 0.9792 - val_loss: 0.3594 - val_accuracy: 0.9167
Epoch 234/500
3/3 [==============================] - 0s 24ms/step - loss: 0.3797 - accuracy: 0.9792 - val_loss: 0.3581 - val_accuracy: 0.9167
Epoch 235/500
3/3 [==============================] - 0s 23ms/step - loss: 0.3784 - accuracy: 0.9792 - val_loss: 0.3568 - val_accuracy: 0.9167
Epoch 236/500
3/3 [==============================] - 0s 24ms/step - loss: 0.3769 - accuracy: 0.9792 - val_loss: 0.3563 - val_accuracy: 0.9167
Epoch 237/500
3/3 [==============================] - 0s 23ms/step - loss: 0.3755 - accuracy: 0.9792 - val_loss: 0.3549 - val_accuracy: 0.9167
Epoch 238/500
3/3 [==============================] - 0s 24ms/step - loss: 0.3742 - accuracy: 0.9792 - val_loss: 0.3535 - val_accuracy: 0.9167
Epoch 239/500
3/3 [==============================] - 0s 21ms/step - loss: 0.3730 - accuracy: 0.9792 - val_loss: 0.3529 - val_accuracy: 0.9167
Epoch 

Epoch 291/500
3/3 [==============================] - 0s 20ms/step - loss: 0.3023 - accuracy: 0.9896 - val_loss: 0.2902 - val_accuracy: 0.9583
Epoch 292/500
3/3 [==============================] - 0s 18ms/step - loss: 0.3007 - accuracy: 0.9896 - val_loss: 0.2890 - val_accuracy: 0.9583
Epoch 293/500
3/3 [==============================] - 0s 18ms/step - loss: 0.3001 - accuracy: 0.9896 - val_loss: 0.2892 - val_accuracy: 0.9583
Epoch 294/500
3/3 [==============================] - 0s 19ms/step - loss: 0.2982 - accuracy: 0.9896 - val_loss: 0.2878 - val_accuracy: 0.9583
Epoch 295/500
3/3 [==============================] - 0s 18ms/step - loss: 0.2971 - accuracy: 0.9896 - val_loss: 0.2870 - val_accuracy: 0.9583
Epoch 296/500
3/3 [==============================] - 0s 20ms/step - loss: 0.2963 - accuracy: 0.9896 - val_loss: 0.2842 - val_accuracy: 0.9583
Epoch 297/500
3/3 [==============================] - 0s 19ms/step - loss: 0.2946 - accuracy: 0.9896 - val_loss: 0.2835 - val_accuracy: 0.9583
Epoch 

In [59]:
###########
# 모델 예측
###########
y_pred = model.predict(x_test)

y_pred[:5]



array([[6.2787323e-04, 1.6920653e-01, 8.3016562e-01],
       [2.0156227e-04, 1.5077306e-01, 8.4902543e-01],
       [9.1394442e-01, 8.5740820e-02, 3.1473665e-04],
       [2.8935794e-03, 3.9096349e-01, 6.0614300e-01],
       [9.6126258e-01, 3.8661670e-02, 7.5802025e-05]], dtype=float32)

In [60]:
y_pred[:5].argmax(axis=1).reshape(-1,1)

array([[2],
       [2],
       [0],
       [2],
       [0]], dtype=int64)

In [61]:
y_test[:5]

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)